`12_langgraph_rag`

# RAG w/ Langgraph

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from pprint import pprint

# 1. Loader (웹문서)
from langchain_community.document_loaders import WebBaseLoader
# pip install beautifulsoup4
from bs4.filter import SoupStrainer

loader = WebBaseLoader(
    web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/', ),
    # 웹페이지 안에서 필요한 정보만 선택하기
    bs_kwargs={
        'parse_only': SoupStrainer(class_=['post-content'])
    }
    # header_template={}
)
docs = loader.load()

# 2. Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
print(len(splitted_docs))

# 3. Embedding Model
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')  # small <-> large

# 4. Vectorstore (지금은 FAISS -> 클라우드-Pinecone)
from langchain_community.vectorstores import FAISS 

vectorstore = FAISS.from_documents(splitted_docs, embedding=embedding)

63


In [10]:
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')

for m in prompt.messages:
    m.pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [23]:
# LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1', temperature=0)

# State
from langchain_core.documents import Document
from typing_extensions import TypedDict, List

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Node
# 검색 노드
def retrieve(state: State):
    # [ Document * 4 ]
    retrieved_docs = vectorstore.similarity_search(state['question'], k=4)

    # 나머지 return 하지 않은 state 항목들은, 알아서 그대로 감 (question, answer 는 알아서 그대로 나감)
    return { 'context': retrieved_docs, }

# 답변 생성노드
def generate(state: State):
    # Document 객체의 필요없는 정보는 다 빼고, 내용에 해당하는 page_content 만 모아서 넘기면 토큰 절약 가능.
    context_str = ''
    for doc in state['context']:
        context_str += doc.page_content + '\n---------------------------\n'

    question_with_context = prompt.invoke({'question': state['question'], 'context': context_str})
    response = llm.invoke(question_with_context)
    return {'answer': response.content}

# Graph
from langgraph.graph import StateGraph, START, END
builder = StateGraph(State)

builder.add_node('retrieve', retrieve)
builder.add_node('generate', generate)

builder.add_edge(START, 'retrieve')
builder.add_edge('retrieve', 'generate')
builder.add_edge('generate', END)

graph = builder.compile()

# 출력
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [24]:
final_state = graph.invoke({'question': '에이전트 시스템에 대해 알려줘!'})

final_state['answer']

'에이전트 시스템은 다양한 자원을 활용해 주어진 작업을 자동으로 수행하는 소프트웨어 구조입니다. 예를 들어, 인터넷 검색, 장기 메모리 관리, 하위 작업 위임(GPT-3.5 기반), 파일 출력 등 여러 기능을 조합해 목표를 달성합니다. 또한, 자신의 행동을 지속적으로 평가하고 개선하며, 효율적으로 작업을 처리하도록 설계되어 있습니다.'

그래프 스트리밍

In [22]:
for message, metadata in graph.stream(
    {"question": "What is Task Decomposition? 한국어로 답해줘"}, stream_mode="messages"
):
    print(message.content, end="")

Task Decomposition(작업 분해)는 복잡한 큰 작업을 더 작고 단순한 여러 단계나 하위 목표로 나누는 과정입니다. 이렇게 하면 각 단계별로 문제를 해결하기 쉬워지고, 전체 작업을 체계적으로 수행할 수 있습니다. 예를 들어, "소설 쓰기"라는 큰 작업을 "줄거리 작성", "등장인물 설정" 등으로 나누는 것이 작업 분해입니다.

## RAG +a
- Metadata 편집
- Query 분석 - 보완

In [30]:
# 메타데이터

# 문서 63개 중 1/3 지점
third = len(splitted_docs) // 3

# metada 에 'section' 추가 중
for idx, doc in enumerate(splitted_docs):
    if idx < third:
        doc.metadata['section'] = 'beginning'
    elif idx < third * 2:
        doc.metadata['section'] = 'middle'
    else:
        doc.metadata['section'] = 'end'

splitted_docs[0].metadata

vectorstore = FAISS.from_documents(splitted_docs, embedding=embedding)

In [31]:
# State를 더 빡빡하게 정의하기 위해, 위에 따로 정의한 클래스 Search

from typing import Literal  # 말 그대로
from typing_extensions import Annotated  # 할 말이 좀 더 있다


class Search(TypedDict):  # StructuredOutput에서 사용하기 위함.
    """Vectorstore Search Query"""
    # 1. 타입, 2. ... -> NOT NULL, 3. 설명
    query: Annotated[str, ..., 'Search query to run']
    section: Annotated[Literal['beginning', 'middle', 'end'], ..., 'Section to query']


class MyState(TypedDict):
    question: str
    query: Search  
    context: List[Document]
    answer: str

In [32]:
# Node
def analyze_query(state: MyState):
    # Search 클래스에 맞춰 사용자 question을 {query, section}로 바꿈
    s_llm = llm.with_structured_output(Search)
    query = s_llm.invoke(state['question'])
    return {'query': query}


def retrieve(state: MyState):
    query = state['query']
    docs = vectorstore.similarity_search(
        query['query'],
        # LLM이 판단한 section과 실제 문서 조각의 section이 맞을 경우에만 검색.
        filter=lambda metadata: metadata.get('section') == query['section']
    )
    return {'context': docs}


def generate(state: MyState):
    # Token 아끼기 위해, 내용만 추려서 문자열로 만들기
    doc_str = ''
    for doc in state['context']:
        doc_str += doc.page_content + '\n==================================\n'
    
    question_with_context = prompt.invoke({'question': state['question'], 'context': doc_str})
    res = llm.invoke(question_with_context)
    return {'answer': res.content}


builder = StateGraph(MyState)
builder.add_node('analyze_query', analyze_query)
builder.add_node('retrieve', retrieve)
builder.add_node('generate', generate)

builder.add_edge(START, 'analyze_query')
builder.add_edge('analyze_query', 'retrieve')
builder.add_edge('retrieve', 'generate')
builder.add_edge('generate', END)

graph = builder.compile()


In [33]:
graph.invoke({'question': '작업분배가 뭐냐'})

{'question': '작업분배가 뭐냐',
 'query': {'query': '작업분배란 무엇인가', 'section': 'beginning'},
 'context': [Document(id='c132dd0f-7702-488d-9f57-c2dd44e8dfd2', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'section': 'beginning'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan bac